### Laboratório 1 - Parte 2

Nesse lab iremos implementar diversas instaciações do modelo vetorial e comparar os resultados com através da métrica MAP.

Instanciações implementadas:

1. Busca binária: Os documentos retornados são aqueles que tem maior match com as palavras da query.
2. TF: Os documentos retornados são aqueles que tem uma maior quantidade das palavras presentes da query.
3. TF-IDF: Similar ao TF mais aplica penalizações as palavras que aparecem em muitos documentos.
4. BM-25 (sem okapi): Incrementa o TF-IDF limitando o número de vezes (k) que uma palavra pode afetar significativamente a posição de um documento.

In [116]:
import csv
import sys
import nltk
from math import log

EMPTY_LIST  = []
TITLE = 1
SUBTITLE = 2
CONTENT = 3
URL = 4
ID = 5
class inverted_index:
    
    BINARY_SEARCH = 1
    TF_SEARCH = 2
    TF_IDF_SEARCH = 3
    BM25_SEARCH = 4

    
    def __init__(self):
        self.invertedIndex = {}
        self.articles = {}
        self.rawData = self.readData()
        TOTAL_DOCUMENTS = len(self.rawData)
        self.buildInvertedIndex(self.rawData)
    
    def readData(self):
        """"Obtains the data from the csv file and returns it as a 
        list of articles, each of them containing title, content and id"""
        data = []
        with open('estadao_noticias_eleicao.csv', 'rt', encoding='utf-8') as csvFile:
            text = [line for line in csv.reader(csvFile, delimiter=',', quotechar='"')]
        return text


    def buildInvertedIndex(self, rawData):
        for article in rawData[1:]:
            
            timestamp, titulo, subtitulo, conteudo, url, idNoticia = article
            idNoticia = int(idNoticia)
            data = nltk.word_tokenize( ' '.join(article[:-1]).lower() )
            
        
            self.addEntriesToIndex(titulo, idNoticia)
            self.addEntriesToIndex(conteudo, idNoticia)
            self.addEntriesToIndex(url, idNoticia)
            
        return self.invertedIndex


    def addEntriesToIndex(self, newsString, articleId):
        for word in newsString.split():
            word = word.lower()
        #    if keyword not in self.invertedIndex:
        #        self.invertedIndex[keyword] = []
        #    self.invertedIndex[keyword].append(articleId)
            if (word in self.invertedIndex) and (articleId in self.invertedIndex[word]):
                self.invertedIndex[word][articleId] += 1
            elif (word in self.invertedIndex) and not (articleId in self.invertedIndex[word]):
                self.invertedIndex[word][articleId] = 1
            else:
                self.invertedIndex[word] = {articleId: 1}
                
    def search(self, query, mode = BINARY_SEARCH ):
        
        query, result = query.lower(), []

        if (mode == self.BINARY_SEARCH):    result = self.binarySearch(query)
        elif (mode == self.TF_SEARCH):      result = self.tfSearch(query)
        elif (mode == self.TF_IDF_SEARCH):  result = self.tfIdfSearch(query)
        elif (mode == self.BM25_SEARCH):    result = self.bm25Search(query)

        return result[:5] if (len(result) > 5) else result
        
    def binarySearch(self, query):
        
        for _word in nltk.word_tokenize(query):
            matchedDocs = {}
            if _word in self.invertedIndex:
                matches = set(self.invertedIndex[_word].keys())
                matchedDocs = matches if len(matchedDocs) == 0 else matchedDocs.intersection(matches)
                
        weightedDocs = [(docId, self.get_bin(docId, query)) for docId in matchedDocs]
 
        return [e[0] for e in sorted(weightedDocs, key= lambda x: x[1], reverse = True)]
    
    
    
    def tfSearch(self, query):
        docs = set([docId for w in query.split() for docId in self.invertedIndex[w].keys() if w in self.invertedIndex])
        docs = [(docId, self.get_tf(docId, query)) for docId in docs ]
        return [e[0] for e in sorted(docs, key= lambda x: x[1], reverse = True)]

    def tfIdfSearch(self, query):
        docs = self.binarySearch(query)
        docs = [(docId, self.get_tf_idf(docId, query)) for docId in docs ]
        return [e[0] for e in sorted(docs, key= lambda x: x[1], reverse = True)]

    def bm25Search(self, query):
        docs = self.binarySearch(query)
        docs = [(docId, self.get_bm25(docId, query)) for docId in docs ]
        return [e[0] for e in sorted(docs, key= lambda x: x[1], reverse = True)]
    
    def get_bin(self, dId, query):

        return sum(1 for w in nltk.word_tokenize(query) if dId in self.invertedIndex[w])
    
    def get_tf(self, dId, query):
        return sum(self.invertedIndex[w][dId] for w in nltk.word_tokenize(query) if dId in self.invertedIndex[w])

    
    def get_tf_idf(self, dId, query):

        out = 0
        
        for w in nltk.word_tokenize(query):
            if dId in self.invertedIndex[w]:
                tf = self.invertedIndex[w][dId]
                idf = log( len(self.rawData) / tf )
                out += query.count(w)*tf*idf
        return out

    def get_bm25(self, dId, query):
        out = 0
        for w in nltk.word_tokenize(query):
            k = 5
            if dId in self.invertedIndex[w]:
                tf = self.invertedIndex[w][dId]
                bmWeight = ((k+1)*tf)/(tf + k)
                idf = log( len(self.rawData) / len(self.invertedIndex[w]) )
                out += query.count(w)*bmWeight*idf
                return out
    
    def getArticles(self, idList):
        articles = []
        for id in idList:
            articles.append(self.articles[id])
        return articles
    
    

In [117]:
idx = inverted_index()

### Observando os resultados

Para avaliar o resultado do nosso modelo vetorial iremos utilizar a metrica mapk para comparar nossos resultados com o do gabarito. 0 é o pior resultado e 1 é o melhor resultado

In [118]:
from metrics import mapk

gabaritoFile = open('gabarito.csv', 'r', encoding='utf-8')
gabaritoLines = [line for line in csv.reader(gabaritoFile, delimiter=',', quotechar='"')]

for query, google, busca_binaria, tf, tfidf, bm25 in gabaritoLines[1:]:
    #busca_binaria = ast.literal_eval(busca_binaria)
    #print(len(busca_binaria))
    val = mapk([eval(busca_binaria)], [idx.search(query)], k=5)
    print("MAP  = {1} para binarySearch com query = {0}".format(query, val))

MAP  = 0.6 para binarySearch com query = segundo turno
MAP  = 0.05 para binarySearch com query = lava jato
MAP  = 0.4 para binarySearch com query = projeto de lei
MAP  = 0.06666666666666667 para binarySearch com query = compra de voto
MAP  = 0.4 para binarySearch com query = ministério público


In [119]:
for query, google, busca_binaria, tf, tfidf, bm25 in gabaritoLines[1:]:
    val = mapk([eval( tf )], [idx.search(query, inverted_index.TF_SEARCH)], k=5)
    print("MAP  = {1} para TF com query = {0}".format(query, val))

MAP  = 0.8 para TF com query = segundo turno
MAP  = 0.52 para TF com query = lava jato
MAP  = 0.42000000000000004 para TF com query = projeto de lei
MAP  = 0.13 para TF com query = compra de voto
MAP  = 0.8 para TF com query = ministério público


In [120]:
for query, google, busca_binaria, tf, tfidf, bm25 in gabaritoLines[1:]:
    val = mapk([eval( tfidf )], [idx.search(query, inverted_index.TF_IDF_SEARCH)], k=5)
    print("MAP  = {1} para TF-IDF com query = {0}".format(query, val))

MAP  = 0.4833333333333333 para TF-IDF com query = segundo turno
MAP  = 0.52 para TF-IDF com query = lava jato
MAP  = 0.05 para TF-IDF com query = projeto de lei
MAP  = 0.3 para TF-IDF com query = compra de voto
MAP  = 0.8 para TF-IDF com query = ministério público


In [121]:
for query, google, busca_binaria, tf, tfidf, bm25 in gabaritoLines[1:]:
    val = mapk([eval( bm25 )], [idx.search(query, inverted_index.BM25_SEARCH)], k=5)
    print("MAP  = {1} para BM25 com query = {0}".format(query, val))

MAP  = 0.0 para BM25 com query = segundo turno
MAP  = 0.8 para BM25 com query = lava jato
MAP  = 0.8 para BM25 com query = projeto de lei
MAP  = 0.8 para BM25 com query = compra de voto
MAP  = 0.8 para BM25 com query = ministério público


### Comparando com os resultados do google

Abaixo podemos comparar os resultados que obtivemos da nossa implementação do modelo veto com os do google.

In [122]:
for query, google, busca_binaria, tf, tfidf, bm25 in gabaritoLines[1:]:
    modes = [inverted_index.BINARY_SEARCH, inverted_index.TF_SEARCH, inverted_index.TF_IDF_SEARCH, inverted_index.BM25_SEARCH]
    for mode in modes:
        val = mapk([eval( google )], [idx.search(query, mode)], k=5)
        print("MAP  = {1} para query = {0}".format(query, val))

MAP  = 0.0 para query = segundo turno
MAP  = 0.0 para query = segundo turno
MAP  = 0.0 para query = segundo turno
MAP  = 0.0 para query = segundo turno
MAP  = 0.0 para query = lava jato
MAP  = 0.04 para query = lava jato
MAP  = 0.0 para query = lava jato
MAP  = 0.2 para query = lava jato
MAP  = 0.0 para query = projeto de lei
MAP  = 0.0 para query = projeto de lei
MAP  = 0.0 para query = projeto de lei
MAP  = 0.16666666666666666 para query = projeto de lei
MAP  = 0.3 para query = compra de voto
MAP  = 0.0 para query = compra de voto
MAP  = 0.0 para query = compra de voto
MAP  = 0.04 para query = compra de voto
MAP  = 0.0 para query = ministério público
MAP  = 0.05 para query = ministério público
MAP  = 0.05 para query = ministério público
MAP  = 0.0 para query = ministério público
